필요한 패키지들을 불러옵니다.

In [1]:
import zipfile

import os
import os.path
from os.path import join
from os import listdir

import numpy as np
import random
from collections import OrderedDict
import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor

from math import log10, sqrt
import skimage.color as sc

대회를 위한 데이터를 다운로드합니다.

In [2]:
!gdown https://drive.google.com/uc?id=1NJlIDC5gVctEHMqgr_WkxOrjNhvVyNpM
!gdown https://drive.google.com/uc?id=1K68hLWgBY0Gj6e3gwuQYspDPeTMgnGqT
!gdown https://drive.google.com/uc?id=1Tfo_UzktBtGSpJfcCcTbYXrhOlBIwwQd

zip_train = zipfile.ZipFile('/content/khuggle_train.zip')
zip_train.extractall('/content/khuggle_train')
 
zip_train.close()

zip_val = zipfile.ZipFile('/content/khuggle_val.zip')
zip_val.extractall('/content/khuggle_val')
 
zip_val.close()

zip_test = zipfile.ZipFile('/content/khuggle_test.zip')
zip_test.extractall('/content/khuggle_test')
 
zip_test.close()

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1NJlIDC5gVctEHMqgr_WkxOrjNhvVyNpM 

Downloading...
From: https://drive.google.com/uc?id=1K68hLWgBY0Gj6e3gwuQYspDPeTMgnGqT
To: /content/khuggle_test.zip
100% 3.19M/3.19M [00:00<00:00, 49.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Tfo_UzktBtGSpJfcCcTbYXrhOlBIwwQd
To: /content/khuggle_val.zip
100% 5.34M/5.34M [00:00<00:00, 31.0MB/s]


FileNotFoundError: ignored

데이터셋과 관련된 함수들을 정의합니다.



In [ ]:
def get_patch(*args, patch_size, scale):
    ih, iw = args[0].shape[:2]

    tp = patch_size  # target patch (HR)
    ip = tp // scale  # input patch (LR)

    ix = random.randrange(0, iw - ip + 1)
    iy = random.randrange(0, ih - ip + 1)
    tx, ty = scale * ix, scale * iy

    ret = [
        args[0][iy:iy + ip, ix:ix + ip, :],
        *[a[ty:ty + tp, tx:tx + tp, :] for a in args[1:]]
    ] 
    return ret


def set_channel(*args, n_channels=3):
    def _set_channel(img):
        if img.ndim == 2:
            img = np.expand_dims(img, axis=2)

        c = img.shape[2]
        if n_channels == 1 and c == 3:
            img = np.expand_dims(sc.rgb2ycbcr(img)[:, :, 0], 2)
        elif n_channels == 3 and c == 1:
            img = np.concatenate([img] * n_channels, 2)

        return img

    return [_set_channel(a) for a in args]


def np2Tensor(*args, rgb_range):
    def _np2Tensor(img):
        np_transpose = np.ascontiguousarray(img.transpose((2, 0, 1)))
        tensor = torch.from_numpy(np_transpose).float()
        tensor.mul_(rgb_range / 255)

        return tensor

    return [_np2Tensor(a) for a in args]


def augment(*args, hflip=True, rot=True):
    hflip = hflip and random.random() < 0.5
    vflip = rot and random.random() < 0.5
    rot90 = rot and random.random() < 0.5

    def _augment(img):
        if hflip: img = img[:, ::-1, :]
        if vflip: img = img[::-1, :, :]
        if rot90: img = img.transpose(1, 0, 2)

        return img

    return [_augment(a) for a in args]

def default_flist_reader(flist):
    imlist = []
    with open(flist, 'r') as rf:
        for line in rf.readlines():
            impath = line.strip()
            imlist.append(impath)

    return imlist


IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP', '.npy'
]


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)


def make_dataset(dir):
    images = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir

    for root, _, fnames in sorted(os.walk(dir)):
        for fname in fnames:
            if is_image_file(fname):
                path = os.path.join(root, fname)
                images.append(path)

    return images


def default_loader(path):
    return Image.open(path).convert('RGB')


class ImageFolder(data.Dataset):

    def __init__(self, root, transform=None, return_paths=False,
                 loader=default_loader):
        imgs = make_dataset(root)
        if len(imgs) == 0:
            raise(RuntimeError("Found 0 images in: " + root + "\n"
                               "Supported image extensions are: " +
                               ",".join(IMG_EXTENSIONS)))

        self.root = root
        self.imgs = imgs
        self.transform = transform
        self.return_paths = return_paths
        self.loader = loader

    def __getitem__(self, index):
        path = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.return_paths:
            return img, path
        else:
            return img

    def __len__(self):
        return len(self.imgs)


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)


def make_dataset(dir):
    images = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir

    for root, _, fnames in sorted(os.walk(dir)):
        for fname in fnames:
            if is_image_file(fname):
                path = os.path.join(root, fname)
                images.append(path)
    return images


class Div2k(data.Dataset):
    def __init__(self, root, phase, train_size, batch_size, patch_size):
        self.scale = 4
        self.root = root
        self.ext = '.png'   # '.png' or '.npy'(default)
        self.train = True if phase == 'train' else False
        self.repeat = 1000 // (train_size // batch_size) # 800 = train dataset size
        self._set_filesystem(self.root)
        self.images_hr, self.images_lr = self._scan()
        self.patch_size = patch_size
        self.train_size = train_size

    def _set_filesystem(self, dir_data):
        self.root = dir_data
        self.dir_hr = os.path.join(self.root, 'khuggle_DIV2K_hr')
        self.dir_lr = os.path.join(self.root, 'khuggle_DIV2K_lr')

    def __getitem__(self, idx):
        lr, hr = self._load_file(idx)
        lr, hr = self._get_patch(lr, hr)
        lr, hr = set_channel(lr, hr, n_channels=3)
        lr_tensor, hr_tensor = np2Tensor(lr, hr, rgb_range=1)
        return lr_tensor, hr_tensor

    def __len__(self):
        if self.train:
            return self.train_size * self.repeat

    def _get_index(self, idx):
        if self.train:
            return idx % self.train_size
        else:
            return idx

    def _get_patch(self, img_in, img_tar):
        patch_size = self.patch_size
        scale = self.scale
        if self.train:
            img_in, img_tar = get_patch(
                img_in, img_tar, patch_size=patch_size, scale=scale)
            img_in, img_tar = augment(img_in, img_tar)
        else:
            ih, iw = img_in.shape[:2]
            img_tar = img_tar[0:ih * scale, 0:iw * scale, :]
        return img_in, img_tar

    def _scan(self):
        list_hr = sorted(make_dataset(self.dir_hr))
        list_lr = sorted(make_dataset(self.dir_lr))
        return list_hr, list_lr

    def _load_file(self, idx):
        idx = self._get_index(idx)
        if self.ext == '.npy':
            lr = npy_loader(self.images_lr[idx])
            hr = npy_loader(self.images_hr[idx])
        else:
            lr = default_loader(self.images_lr[idx])
            hr = default_loader(self.images_hr[idx])
        return lr, hr


class DatasetFromFolderVal(data.Dataset):
    def __init__(self, hr_dir, lr_dir, upscale):
        super(DatasetFromFolderVal, self).__init__()
        self.hr_filenames = sorted([join(hr_dir, x) for x in listdir(hr_dir) if is_image_file(x)])
        self.lr_filenames = sorted([join(lr_dir, x) for x in listdir(lr_dir) if is_image_file(x)])
        self.upscale = upscale

    def __getitem__(self, index):
        input = load_image(self.lr_filenames[index])
        target = load_image(self.hr_filenames[index])
        input = np2tensor()(input)
        target = np2tensor()(img_modcrop(target, self.upscale))

        return input, target

    def __len__(self):
        return len(self.lr_filenames)

성능 측정 및 각종 유틸리티 관련 함수들을 정의합니다.

In [ ]:
def compute_psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr


def shave(im, border):
    border = [border, border]
    im = im[border[0]:-border[0], border[1]:-border[1], ...]
    return im


def modcrop(im, modulo):
    sz = im.shape
    h = np.int32(sz[0] / modulo) * modulo
    w = np.int32(sz[1] / modulo) * modulo
    ims = im[0:h, 0:w, ...]
    return ims


def get_list(path, ext):
    return [os.path.join(path, f) for f in os.listdir(path) if f.endswith(ext)]


def convert_shape(img):
    img = np.transpose((img * 255.0).round(), (1, 2, 0))
    img = np.uint8(np.clip(img, 0, 255))
    return img


def tensor2np(tensor, out_type=np.uint8, min_max=(0, 1)):
    tensor = tensor.float().cpu().clamp_(*min_max)
    tensor = (tensor - min_max[0]) / (min_max[1] - min_max[0])  # to range [0, 1]
    img_np = tensor.numpy()
    img_np = np.transpose(img_np, (1, 2, 0))
    if out_type == np.uint8:
        img_np = (img_np * 255.0).round()

    return img_np.astype(out_type)


def convert2np(tensor):
    return tensor.cpu().mul(255).clamp(0, 255).byte().squeeze().permute(1, 2, 0).numpy()


def adjust_learning_rate(optimizer, epoch, step_size, lr_init, gamma):
    factor = epoch // step_size
    lr = lr_init * (gamma ** factor)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def img_modcrop(image, modulo):
    sz = image.size
    w = np.int32(sz[0] / modulo) * modulo
    h = np.int32(sz[1] / modulo) * modulo
    out = image.crop((0, 0, w, h))
    return out


def np2tensor():
    return Compose([
        ToTensor(),
    ])


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".bmp", ".png", ".jpg"])


def load_image(filepath):
    return Image.open(filepath).convert('RGB')


def default_loader(path):
    return cv2.imread(path, cv2.IMREAD_UNCHANGED)[:, :, [2, 1, 0]]


def npy_loader(path):
    return np.load(path)


def load_state_dict(path):
    state_dict = torch.load(path)
    new_state_dcit = OrderedDict()
    for k, v in state_dict.items():
        if 'module' in k:
            name = k[7:]
        else:
            name = k
        new_state_dcit[name] = v
    return 

이번 대회에서 사용할 초해상화 모델인 IMDN(Information Multi-distillation Network)의 구조를 이루는 함수들을 정의합니다.

In [ ]:
def conv_layer(in_channels, out_channels, kernel_size, stride=1, dilation=1, groups=1, bias=True):
    padding = int((kernel_size - 1) / 2) * dilation
    return nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=padding, bias=bias, dilation=dilation,
                     groups=groups)


def norm(norm_type, nc):
    norm_type = norm_type.lower()
    if norm_type == 'batch':
        layer = nn.BatchNorm2d(nc, affine=True)
    elif norm_type == 'instance':
        layer = nn.InstanceNorm2d(nc, affine=False)
    else:
        raise NotImplementedError('normalization layer [{:s}] is not found'.format(norm_type))
    return layer


def pad(pad_type, padding):
    pad_type = pad_type.lower()
    if padding == 0:
        return None
    if pad_type == 'reflect':
        layer = nn.ReflectionPad2d(padding)
    elif pad_type == 'replicate':
        layer = nn.ReplicationPad2d(padding)
    else:
        raise NotImplementedError('padding layer [{:s}] is not implemented'.format(pad_type))
    return layer


def get_valid_padding(kernel_size, dilation):
    kernel_size = kernel_size + (kernel_size - 1) * (dilation - 1)
    padding = (kernel_size - 1) // 2
    return padding


def conv_block(in_nc, out_nc, kernel_size, stride=1, dilation=1, groups=1, bias=True,
               pad_type='zero', norm_type=None, act_type='relu'):
    padding = get_valid_padding(kernel_size, dilation)
    p = pad(pad_type, padding) if pad_type and pad_type != 'zero' else None
    padding = padding if pad_type == 'zero' else 0

    c = nn.Conv2d(in_nc, out_nc, kernel_size=kernel_size, stride=stride, padding=padding,
                  dilation=dilation, bias=bias, groups=groups)
    a = activation(act_type) if act_type else None
    n = norm(norm_type, out_nc) if norm_type else None
    return sequential(p, c, n, a)


def activation(act_type, inplace=True, neg_slope=0.05, n_prelu=1):
    act_type = act_type.lower()
    if act_type == 'relu':
        layer = nn.ReLU(inplace)
    elif act_type == 'lrelu':
        layer = nn.LeakyReLU(neg_slope, inplace)
    elif act_type == 'prelu':
        layer = nn.PReLU(num_parameters=n_prelu, init=neg_slope)
    else:
        raise NotImplementedError('activation layer [{:s}] is not found'.format(act_type))
    return layer


class ShortcutBlock(nn.Module):
    def __init__(self, submodule):
        super(ShortcutBlock, self).__init__()
        self.sub = submodule

    def forward(self, x):
        output = x + self.sub(x)
        return output


def mean_channels(F):
    assert(F.dim() == 4)
    spatial_sum = F.sum(3, keepdim=True).sum(2, keepdim=True)
    return spatial_sum / (F.size(2) * F.size(3))


def stdv_channels(F):
    assert(F.dim() == 4)
    F_mean = mean_channels(F)
    F_variance = (F - F_mean).pow(2).sum(3, keepdim=True).sum(2, keepdim=True) / (F.size(2) * F.size(3))
    return F_variance.pow(0.5)


def sequential(*args):
    if len(args) == 1:
        if isinstance(args[0], OrderedDict):
            raise NotImplementedError('sequential does not support OrderedDict input.')
        return args[0]
    modules = []
    for module in args:
        if isinstance(module, nn.Sequential):
            for submodule in module.children():
                modules.append(submodule)
        elif isinstance(module, nn.Module):
            modules.append(module)
    return nn.Sequential(*modules)


class CCALayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(CCALayer, self).__init__()

        self.contrast = stdv_channels
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv_du = nn.Sequential(
            nn.Conv2d(channel, channel // reduction, 1, padding=0, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel // reduction, channel, 1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.contrast(x) + self.avg_pool(x)
        y = self.conv_du(y)
        return x * y


class IMDModule(nn.Module):
    def __init__(self, in_channels, distillation_rate=0.25):
        super(IMDModule, self).__init__()
        self.distilled_channels = int(in_channels * distillation_rate)
        self.remaining_channels = int(in_channels - self.distilled_channels)
        self.c1 = conv_layer(in_channels, in_channels, 3)
        self.c2 = conv_layer(self.remaining_channels, in_channels, 3)
        self.c3 = conv_layer(self.remaining_channels, in_channels, 3)
        self.c4 = conv_layer(self.remaining_channels, self.distilled_channels, 3)
        self.act = activation('lrelu', neg_slope=0.05)
        self.c5 = conv_layer(in_channels, in_channels, 1)
        self.cca = CCALayer(self.distilled_channels * 4)

    def forward(self, input):
        out_c1 = self.act(self.c1(input))
        distilled_c1, remaining_c1 = torch.split(out_c1, (self.distilled_channels, self.remaining_channels), dim=1)
        out_c2 = self.act(self.c2(remaining_c1))
        distilled_c2, remaining_c2 = torch.split(out_c2, (self.distilled_channels, self.remaining_channels), dim=1)
        out_c3 = self.act(self.c3(remaining_c2))
        distilled_c3, remaining_c3 = torch.split(out_c3, (self.distilled_channels, self.remaining_channels), dim=1)
        out_c4 = self.c4(remaining_c3)
        out = torch.cat([distilled_c1, distilled_c2, distilled_c3, out_c4], dim=1)
        out_fused = self.c5(self.cca(out)) + input
        return out_fused
    
def pixelshuffle_block(in_channels, out_channels, upscale_factor=2, kernel_size=3, stride=1):
    conv = conv_layer(in_channels, out_channels * (upscale_factor ** 2), kernel_size, stride)
    pixel_shuffle = nn.PixelShuffle(upscale_factor)
    return sequential(conv, pixel_shuffle)

IMDN 네트워크를 정의합니다. 자세한 내용은 논문 (https://arxiv.org/pdf/1909.11856v1.pdf) 를 참고해주세요.


In [ ]:
class IMDN(nn.Module):
    def __init__(self, in_nc=3, nf=64, num_modules=6, out_nc=3, upscale=4):
        super(IMDN, self).__init__()

        self.fea_conv = conv_layer(in_nc, nf, kernel_size=3)

        # IMDBs
        self.IMDB1 = IMDModule(in_channels=nf)
        self.IMDB2 = IMDModule(in_channels=nf)
        self.IMDB3 = IMDModule(in_channels=nf)
        self.IMDB4 = IMDModule(in_channels=nf)
        self.IMDB5 = IMDModule(in_channels=nf)
        self.IMDB6 = IMDModule(in_channels=nf)
        self.c = conv_block(nf * num_modules, nf, kernel_size=1, act_type='lrelu')

        self.LR_conv = conv_layer(nf, nf, kernel_size=3)

        upsample_block = pixelshuffle_block
        self.upsampler = upsample_block(nf, out_nc, upscale_factor=upscale)


    def forward(self, input):
        out_fea = self.fea_conv(input)
        out_B1 = self.IMDB1(out_fea)
        out_B2 = self.IMDB2(out_B1)
        out_B3 = self.IMDB3(out_B2)
        out_B4 = self.IMDB4(out_B3)
        out_B5 = self.IMDB5(out_B4)
        out_B6 = self.IMDB6(out_B5)

        out_B = self.c(torch.cat([out_B1, out_B2, out_B3, out_B4, out_B5, out_B6], dim=1))
        out_lr = self.LR_conv(out_B) + out_fea
        output = self.upsampler(out_lr)
        return output

모델 훈련을 시작합니다.

In [ ]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu100/torch_nightly.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/nightly/cu100/torch_nightly.html
     |████████████████████████████████| 790.0 MB 21 kB/s 


In [ ]:
print('''

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   @@@@@   .@@.  #@@@@@@&   @@@*  *@@@@@@   &@@@@@@@       .@@@@@@@@        @@@@*  /@@@@@@@@         @@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   @@@.   @@@@*  /@@@@@@@   @@@#  .@@@@@@   #@@@@    /@@@@% @@@@@    .@@@@@ .@@@#   @@@@@@@@   @@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   @%   @@@@@@#   @@@@@@@   @@@@   @@@@@@/  ,@@&   @@@@@@@@@@@@@   &@@@@@@@@@@@@@   @@@@@@@@   &@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@      ,@@@@@@@@             @@@@   @@@@@@&   @@,  .@@@@      @@#   @@@@      /@@@   @@@@@@@@         @@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@   &@   .@@@@@@   @@@@@@@   &@@@   @@@@@@&   @@&   @@@@@@@   %@@   &@@@@@@#   @@@   @@@@@@@@(  .@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@,  *@@@   .@@@@   @@@@@@@.  (@@@@   #@@@&   #@@@@    @@@@@*  *@@@    %@@@@&   @@@   @@@@@@@@&   @@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@%   @@@@@    @@   &@@@@@@(  .@@@@@&       *@@@@@@@@(        *@@@@@@&         @@@@         *@@         @@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@          @@@@@&          @@@@@@@.         @@@@@@           @@@@@@@@@@@@@.  (@@@@@@@&   @@@@@@,  /@@@@@,        #@@*  ,@@@@   ,@@@   @@@@*  *@
@@@@@   ,@@@@@@@@@@@@   ,@@@@@@    @@@.   @@@@@@(   @@@@   @@@@@@    @@@@@@@@@@@(  .@@@@@@@@   @@@@@@#   @@@.   &@@@@@@@@@&   @@.   @@@@@,  &@@@%  ,@@
@@@   #@@@@@@@@@@@@.  ,@@@@@@@@,  .@&   @@@@@@@@@   @@@   @@@@@@@(   @@@@@@@@@@&   @@@@@@@@   @@@@@@&   @@#   @@@@@@@@@@@&   #   @@@@@@@@  @@@@@  .@@@
@@   %@@@%      @@.   @@@@@@@@&   @@   @@@@@@@@@   @@@   @@@@@@@&   @@@@@@@@@@@   @@@@@@@@   @@@@@@@   @@(   @@@@@@@@@@@@   *   @@@@@@@@  (@@@@   @@@@
@&   &@@@@@@   @@@   /@@@@@@@   /@@.   @@@@@@@/   @@@   %@@@@@@   *@@@@@@@@@@@   @@@@@@@@,  .@@@@@&   @@@    @@@@@@@@@@@   @@.   @@@@@@@@%@@@@#@@@@@@@
@@            @@@@%           &@@@@@           *@@@@*           @@@@@@@@@@@@@         &@@*          @@@@@%          @@@   @@@@.   &@@@   @@@@   &@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

''')


torch.backends.cudnn.benchmark = True

# random seed
seed = 0#random.randint(1, 10000)
print("Ramdom Seed: ", seed)
random.seed(seed)
torch.manual_seed(seed)

cuda = True
device = torch.device('cuda' if cuda else 'cpu')

step_size = 200
lr = 2e-4
gamma = 0.5

nEpochs = 1000

print("===> Loading datasets")

trainset = Div2k(root='/content/khuggle_train', phase='train', train_size=800, batch_size=16, patch_size=192)
testset = DatasetFromFolderVal(hr_dir="/content/khuggle_val/Set14_hr/",
                               lr_dir="/content/khuggle_val/Set14_lr/", upscale=4)

training_data_loader = DataLoader(dataset=trainset, num_workers=2, batch_size=16, shuffle=True, pin_memory=True, drop_last=True)
testing_data_loader = DataLoader(dataset=testset, num_workers=2, batch_size=1, shuffle=False)

print("===> Building models")

model = IMDN(upscale=4)
l1_criterion = nn.L1Loss()

print("===> Setting GPU")

if cuda:
    model = model.to(device)
    l1_criterion = l1_criterion.to(device)

# pre-trained 된 모델이 있을 경우 None을 주석 처리하고, pre-trained 된 모델의 경로를 작성합니다.

pretrained = None # '/models/imdn.pth'

if pretrained:

    if os.path.isfile(pretrained):
        print("===> loading models '{}'".format(pretrained))
        checkpoint = torch.load(pretrained)
        new_state_dcit = OrderedDict()
        for k, v in checkpoint.items():
            if 'module' in k:
                name = k[7:]
            else:
                name = k
            new_state_dcit[name] = v
        model_dict = model.state_dict()
        pretrained_dict = {k: v for k, v in new_state_dcit.items() if k in model_dict}

        for k, v in model_dict.items():
            if k not in pretrained_dict:
                print(k)
        model.load_state_dict(pretrained_dict, strict=True)

    else:
        print("===> no models found at '{}'".format(pretrained))

print("===> Setting Optimizer")

optimizer = optim.Adam(model.parameters(), lr=lr)

def train(epoch):
    model.train()
    adjust_learning_rate(optimizer, epoch, step_size, lr, gamma)

    print('epoch =', epoch, 'lr = ', optimizer.param_groups[0]['lr'])

    for iteration, (lr_tensor, hr_tensor) in enumerate(training_data_loader, 1):

        if cuda:
            lr_tensor = lr_tensor.to(device)  # ranges from [0, 1]
            hr_tensor = hr_tensor.to(device)  # ranges from [0, 1]

        optimizer.zero_grad()
        sr_tensor = model(lr_tensor)
        loss_l1 = l1_criterion(sr_tensor, hr_tensor)
        loss_sr = loss_l1

        loss_sr.backward()
        optimizer.step()

        if iteration % 100 == 0:
            print("===> Epoch[{}]({}/{}): Loss_l1: {:.5f}".format(epoch, iteration, len(training_data_loader), loss_l1.item()))


def valid():
    model.eval()

    avg_psnr = 0
    for batch in testing_data_loader:
        lr_tensor, hr_tensor = batch[0], batch[1]
        if cuda:
            lr_tensor = lr_tensor.to(device)
            hr_tensor = hr_tensor.to(device)
        
        with torch.no_grad():
            pre = model(lr_tensor)
        sr_img = tensor2np(pre.detach()[0])
        gt_img = tensor2np(hr_tensor.detach()[0])
        crop_size = 4 # scale
        cropped_sr_img = shave(sr_img, crop_size)
        cropped_gt_img = shave(gt_img, crop_size)
        
        im_label = cropped_gt_img
        im_pre = cropped_sr_img

        avg_psnr += compute_psnr(im_pre, im_label)
    
    print("===> Valid. psnr: {:.4f}".format(avg_psnr / len(testing_data_loader)))


def save_checkpoint(epoch):
    model_folder = "checkpoint/"
    model_out_path = model_folder + "epoch_{}.pth".format(epoch)
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)
    torch.save(model.state_dict(), model_out_path)
    
    print("===> Checkpoint saved to {}".format(model_out_path))

def print_network(net):
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    
    print(net)
    print('Total number of parameters: %d' % num_params)


print("===> Training")
print_network(model)

for epoch in range(1, nEpochs + 1):
    valid()
    train(epoch)
    save_checkpoint(epoch)

In [ ]:
cuda = True
device = torch.device('cuda' if cuda else 'cpu')

test_lr_folder = '/home/hallasan/junghun/EDSR-PyTorch/dataset/benchmark/Set5/LR_bicubic/X4/'
output_folder = './khuggle_output/'

lrs = os.listdir(test_lr_folder[:-1])

filelist = get_list(test_lr_folder, ext='.png')

# 추론에 사용할 모델 가중치 불러오기
checkpoint = '/home/hallasan/junghun/checkpoint/epoch_4.pth'

model = IMDN(upscale=4)
model.load_state_dict(torch.load(checkpoint), strict=True)

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

for i, imname in enumerate(filelist):
    print(imname)
    im_l = cv2.imread(imname, cv2.IMREAD_COLOR)[:, :, [2, 1, 0]]  # BGR to RGB
    if len(im_l.shape) < 3:
        im_l = im_l[..., np.newaxis]
        im_l = np.concatenate([im_l] * 3, 2)
    im_input = im_l / 255.0
    im_input = np.transpose(im_input, (2, 0, 1))
    im_input = im_input[np.newaxis, ...]
    im_input = torch.from_numpy(im_input).float()

    if cuda:
        model = model.to(device)
        im_input = im_input.to(device)

    with torch.no_grad():
        start.record()
        out = model(im_input)
        end.record()
        torch.cuda.synchronize()

    out_img = tensor2np(out.detach()[0])

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cv2.imwrite(output_folder + lrs[i][:-4], out_img[:, :, [2, 1, 0]])